In [1]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark.sql import SparkSession

In [5]:
conf = SparkConf().set("spark.cores.max", "16") \
    .set("spark.driver.memory", "16g") \
    .set("spark.executor.memory", "16g") \
    .set("spark.executor.memory_overhead", "16g") \
    .set("spark.driver.maxResultsSize", "0") \
    .set("spark.sql.shuffle.partitions","300")

sc = SparkContext(appName="testRDD", conf=conf)
spark = SparkSession(sc)

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [8]:
from pyspark.sql.functions import isnan, count, col

In [12]:
#Read PaperAuthorAffiliations Data
PapAuthAffiliations_df = spark.read.options(header = True,delimiter=',',inferSchema=True ).csv("PaperAuthorAffilitions.csv")

In [13]:
PapAuthAffiliations_df.show(30)

+----------+----------+-------------+--------------------+
|   PaperID|  AuthorId|AffiliationId|AuthorSequenceNumber|
+----------+----------+-------------+--------------------+
|2514067917|2165266595|   1334877674|                   1|
|2514067917|2334513227|   1334877674|                   2|
|2514067917|2159674999|   1334877674|                   3|
|2514067917|2133135916|   1334877674|                   4|
|2273582415| 546406812|    205783295|                   4|
|2273582415|2109773566|    205783295|                   3|
|2273582415|2163045884|    205783295|                   2|
|2273582415|1898881442|    205783295|                   1|
|1963479517|2296113232|    142823887|                   1|
|1963479517|2506097535|    134161618|                   3|
|1963479517|2167258348|    134161618|                   2|
|2521463243|2160003181|    169521973|                   8|
|2521463243|2435365629|    192454743|                   5|
|2521463243|2143447455|    192454743|                   

In [14]:
#Read Author Data
Author_df = spark.read.options(header = 'True',delimiter=',',inferSchema=True ).csv("Author.csv")

In [17]:
Author_df.show(2)

+----------+----------+-----------------+--------------+------------------+-------------+----------------+
|  AuthorId|AuthorRank|AutNormalizedName|AutDisplayName|AutLastAffiliation|AutPaperCount|AutCitationCount|
+----------+----------+-----------------+--------------+------------------+-------------+----------------+
|2136881011|     18252| godson r e e ana|    Godson Ana|           9446555|           39|             166|
|2763836685|     19705|     frank p bell| Frank P. Bell|         125749732|            1|               1|
+----------+----------+-----------------+--------------+------------------+-------------+----------------+
only showing top 2 rows



In [18]:
drop_cols = ["AutNormalizedName","AutDisplayName","AutLastAffiliation"]
Author_df = Author_df.drop(*drop_cols)

In [19]:
#Joining Paper and Journal Data
df = Author_df.join(PapAuthAffiliations_df, on=['AuthorId'], how='inner')

In [20]:
PapAuthAffiliations_df.unpersist()

DataFrame[PaperID: bigint, AuthorId: bigint, AffiliationId: bigint, AuthorSequenceNumber: int]

In [21]:
Author_df.unpersist()

DataFrame[AuthorId: bigint, AuthorRank: int, AutPaperCount: int, AutCitationCount: int]

In [22]:
#df.printSchema()

In [23]:
df.count()

223953283

In [24]:
df.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('AuthAndPapAuthAffMerged.csv',header = 'true')